In [2]:
import sys
sys.path.insert(0, './../')

In [3]:
import dask.dataframe as dd
from dask import delayed
from distributed import Client
from distributed import progress

from core.data_acquisition import TaxiData

client = Client()
client

Client Scheduler: tcp://127.0.0.1:38537 Dashboard: http://127.0.0.1:8787,Cluster Workers: 8 Cores: 8 Memory: 16.71 GB


In [11]:
links = [data_link for data_link in TaxiData.yield_s3_links('2015-12')]
dfs = [delayed(TaxiData.load_from_url)(url, n_bytes=6400) for url in links]
dfs = [delayed(TaxiData.clean_df)(df) for df in dfs]


In [27]:
import dask.bag as db

dfs = db.from_url([link.replace('https', 'https') for link in links])

In [35]:
dfs = dd.read_table([link.replace('https://s3.amazonaws.com/', 's3://') for link in links])

FileNotFoundError: nyc-tlc/trip+data/yellow_tripdata_2017-01.csv

In [4]:
df = dd.read_csv('s3://nyc-tlc/trip data/yellow_*_2017-01.csv', blocksize=6400000)

In [8]:
df.count().compute()

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7f4be72d3268>, address='tcp://127.0.0.1:45509')
Traceback (most recent call last):
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.5/site-packages/tornado/ioloop.py", line 605, in _run_callback
    ret = callback()
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.5/site-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.5/site-packages/distributed/scheduler.py", line 950, in remove_worker
    self.transitions(recommendations)
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.5/site-packages/distributed/scheduler.py", line 3051, in transitions
    new = self.transition(key, finish)
  File "/home/milesg/anaconda/envs/nyc-taxi-aws/lib/python3.5/site-packages/distributed/scheduler.py", line 3007, in transition
    recommendations = func(key, *args

KeyboardInterrupt: 

In [6]:
df.npartitions

134